In [2]:
using DelimitedFiles

function read_points(filename)
    raw = read(filename, String)

    # Remove the enclosing double braces {{ ... }}
    inner = raw[3:end-2]          # drop first two chars "{{" and last "}}"

    # Split into individual vector strings
    vecs = split(inner, "},{")

    # Parse each vector
    points = [parse.(Float64, split(replace(v, ['{','}'] => ""), ",")) for v in vecs]

    # Convert to a matrix (Nx3)
    return reduce(vcat, (p' for p in points))
end

pts = read_points("A_sensors_data.txt")


using LinearAlgebra
function distance_table(pts)
    n = size(pts,1)
    D = zeros(Float64, n, n)
    for i in 1:(n-1)
        for j in (i+1):n
            D[i,j] = D[j, i] = norm(pts[i, :]-pts[j, :])
        end
    end
    return D
end
function VR(r, D)
    edges = Tuple{Int,Int}[]
    n=size(D,1)
    for i in 1:(n-1)
        for j in (i+1):n
            if D[i,j] ≤ r 
                push!(edges, (i,j))
            end
        end
    end
    return edges
end

VR (generic function with 1 method)

In [3]:
#my hw1 code for connected components
function edgedictionary(V, E)
    n = V[end]
    # set up the dictionary (empty sets of k-tuples in each dimension)
    edge_dict = Dict(k => Set{Int}() for k in 1:n)
    for (v1, v2) in E
        push!(edge_dict[v1], v2)
        push!(edge_dict[v2], v1)
    end
    return edge_dict
end

function component_of(edge_dict, vertex, visited=Set{Int}())
    push!(visited, vertex)
    component = [vertex]

    for neighbor in edge_dict[vertex]
        if !(neighbor in visited)
            append!(component, component_of(edge_dict, neighbor, visited))
        end
    end
    return sort(component)
end

function findcomponents(V, E)
    edge_dict = edgedictionary(V, E)

    visited = Set{Int}()
    components = []

    for vertex in V
        if !(vertex in visited)
            push!(components, component_of(edge_dict, vertex, visited))
        end
    end
    return components
end

findcomponents (generic function with 1 method)

In [4]:
function is_connected(V,E; exclude=Set{Int}())
    if isempty(exclude)
        comp = findcomponents(V,E)
    else
        E=[e for e in E if !(e[1] in exclude || e[2] in exclude)]
        comp = findcomponents(V,E)
        comp =[c for c in comp if all(x -> !(x in exclude), c)]
    end
    return (length(comp) == 1)
end

function find_r(pts; steps = 1000)
    D = distance_table(pts) 
    V = collect(1:(size(pts,1)))
    lo = 0.0
    hi = maximum(D)
    for _ in 1:steps
        r = (lo + hi) / 2
        edges = VR(r,D)
        if is_connected(V, edges)
            hi = r
        else
            lo = r
        end
    end
    return (lo + hi) / 2
end

find_r (generic function with 1 method)

In [5]:
r=find_r(pts)


0.4965033379263023

r=
0.4965033379263023


In [6]:
r=0.7

0.7

In [7]:
D=distance_table(pts)
V = collect(1:(size(pts,1)))
E=VR(r,D)
#is_connected(V,E,exclude=Set([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]))

172-element Vector{Tuple{Int64, Int64}}:
 (1, 2)
 (1, 3)
 (1, 4)
 (1, 5)
 (1, 6)
 (1, 7)
 (1, 8)
 (1, 9)
 (1, 10)
 (2, 3)
 (2, 4)
 (2, 5)
 (2, 6)
 ⋮
 (48, 49)
 (48, 53)
 (48, 54)
 (49, 50)
 (49, 54)
 (50, 51)
 (50, 54)
 (51, 52)
 (51, 54)
 (52, 53)
 (52, 54)
 (53, 54)

This must be run in julia and not jupyter as display doesnt work in notebook

In [8]:
using WGLMakie, ColorTypes

function plot_sphere_with_points_web(pts, edges)
    fig = Figure(size=(800, 800))
    ax = Axis3(fig[1,1], aspect=:data)

    # Sphere mesh
    θ = range(0, 2π, length=80)
    φ = range(0, π, length=40)
    x = [sin(phi)*cos(th) for phi in φ, th in θ]
    y = [sin(phi)*sin(th) for phi in φ, th in θ]
    z = [cos(phi) for phi in φ, th in θ]

    # Make sphere transparent
    surface!(ax, x, y, z, color=RGBA(0.5, 0.7, 1.0, 0.2), transparency=true)

    # Points
    scatter!(ax, pts[:,1], pts[:,2], pts[:,3], color=RGBA(1,0,0,1), markersize=15)

    # Edges
    for (i,j) in edges
        lines!(ax, [pts[i,1], pts[j,1]], [pts[i,2], pts[j,2]], [pts[i,3], pts[j,3]], color=:black, linewidth=2)
    end

    fig
end

fig = plot_sphere_with_points_web(pts, E)
fig  

To program Check we will use miniballs, and bacause wi will be calculating check alot it is best we precomute them


In [9]:
using BoundingSphere
function MiniBall_edges(points)
    n = size(points,1)
    edges = Dict{Tuple{Int,Int}, Float64}()
    for i in 1:n-1, j in i+1:n
        edges[(i,j)] = norm(points[i,:] - points[j,:]) / 2
    end
    return edges
end
function MiniBall_triangles(points)
    n = size(points,1)
    tris = Dict{NTuple{3,Int}, Float64}()

    for i in 1:n-2, j in i+1:n-1, k in j+1:n
        tris[(i,j,k)] = boundingsphere([
            Vector(points[i,:]),
            Vector(points[j,:]),
            Vector(points[k,:])
        ])[2]
    end
    return tris
end
function MiniBall_tetrahedra(points)
    n = size(points,1)
    tets = Dict{NTuple{4,Int}, Float64}()

    for i in 1:n-3, j in i+1:n-2, k in j+1:n-1, l in k+1:n
        tets[(i,j,k,l)] = boundingsphere([
            Vector(points[i,:]),
            Vector(points[j,:]),
            Vector(points[k,:]),
            Vector(points[l,:])
        ])[2]
    end
    return tets
end


MiniBall_tetrahedra (generic function with 1 method)

In [10]:

#cech already returns a dim_dict
function cech(R, points, MB_edges, MB_triangles, MB_tetrahedra)
    n = size(points, 1)
    simplices = Dict((k-1) => Vector{NTuple{k, Int}}() for k in 1:4)
    simplices[0] = [(i,) for i in 1:n]
    simplices[1] = [(i,j) for ((i,j),r) in MB_edges if r ≤ R]
    simplices[2] = [(i,j,k) for ((i,j,k),r) in MB_triangles if r ≤ R]
    simplices[3] = [(i,j,k,l) for ((i,j,k,l),r) in MB_tetrahedra if r ≤ R]
    return simplices
end


cech (generic function with 1 method)

In [11]:
MB_edges=MiniBall_edges(pts)
MB_triangles=MiniBall_triangles(pts)
MB_tetrahedra=MiniBall_tetrahedra(pts)
cech(0.4, pts, MB_edges, MB_triangles, MB_tetrahedra)

Dict{Int64, Vector} with 4 entries:
  0 => [(1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,)  …  (45,), …
  2 => [(1, 3, 4), (23, 24, 35), (23, 34, 46), (3, 11, 22), (23, 35, 36), (31, …
  3 => [(11, 21, 22, 34), (1, 2, 3, 9), (1, 2, 6, 7), (3, 10, 11, 22), (3, 4, 1…
  1 => [(47, 52), (30, 41), (26, 39), (21, 34), (11, 23), (17, 18), (1, 10), (4…

In [12]:
using Mods, Primes, LinearAlgebra

function simplex_boundary(sx)
    # determine the codimension 1 simplices in the boundary of the simplex sx
    # 'remove' one entry of a tuple at each specific index by splicing the corresponding subtuples
    cd1faces = ([(sx[1:k-1]..., sx[k+1:end]...) for k in eachindex(sx)])
    return cd1faces
end

# This function returns the n-th boundary matrix Dn with coefficients Z/pZ or R of a simplicial complex given as a dimension_dict.
# (p should be a prime. For convenience, p = 0 means R.)
function boundary_matrix(dimension_dict, n, p = 0)
    # check if p is 0 or prime
    if p != 0 && !isprime(p)
        throw(DomainError(p, "p should be 0 or a prime."))
    end
    # cases n = 0 and n = dim+1 should be treated separately
    if n == 0
        Dn = zeros(p==0 ? Float64 : Mod{p}, 1, length(dimension_dict[n]))
    elseif n == maximum(keys(dimension_dict))+1
        Dn = zeros(p==0 ? Float64 : Mod{p}, length(dimension_dict[n-1]), 1)
    else
        Dn = zeros(p==0 ? Float64 : Mod{p}, length(dimension_dict[n-1]), length(dimension_dict[n]))
        # otherwise, just replace the zeroes with 1's or -1's at appropriate places
        for j in eachindex(dimension_dict[n])
            boundary = simplex_boundary(dimension_dict[n][j])
            for i in eachindex(boundary)
                Dn[findfirst(==(boundary[i]), dimension_dict[n-1]), j] = -(-1)^i
            end
        end
    end
    return Dn
end


# determine the mod p Betti numbers of a simplicial complex given via its maximal simplices
function betti_numbers(dimension_dict, p)
    # build the 'dimension dictionary' of a simplicial complex given via its maximal simplices
    #dimension_dict = dim_dict_scx(max_scx)
    # determine the dimension of max_scx
    d = maximum(keys(dimension_dict))
    # initialize the vector of Betti numbers
    betti = []
    # the current (first) boundary matrix
    Dk1 = boundary_matrix(dimension_dict, 0, p)
    # determine the Betti numbers from two consecutive boundary matrices
    for k in 1:(d)
        # the next boundary matrix
        Dk = boundary_matrix(dimension_dict, k, p)
        # perform simultaneous reduction on D_{k-1} and D_k 
        # and store the corresponding Betti number
        push!(betti, simultaneous_reduce(Dk1, Dk))
        # use Dk as the current (first) boundary matrix
        Dk1 = Dk
    end
    return betti
end


function simultaneous_reduce(A, B)
    # throw a 'wrong size' error
    if size(A)[2] != size(B)[1]
        throw(DimensionMismatch("cols(A) != rows(B)"))
    end
    # store the number of rows and columns of A 
    n_rows, n_cols = size(A)
    # perform a column reduction on A and do the inverse operations on rows of B
    i, j = 1, 1
    last_nonzero_col = 0
    while i <= n_rows && j <= n_cols
        # check if A[i, j] can be used as a column pivot ...
        if A[i, j] == 0
            # if not, find a column pivot in the current row
            nonzero_col = j+1
            while nonzero_col <= n_cols && A[i, nonzero_col] == 0
                nonzero_col += 1;
            end
            # if there are only zeros in the current row to the right
            if nonzero_col == n_cols+1
                # go to a new row
                i += 1
                continue
            end
            # otherwise swap the columns of A ...
            A[:, [j nonzero_col]] = A[:, [nonzero_col j]]
            # ... and corresponding rows of B
            B[[j nonzero_col], :] = B[[nonzero_col j], :]
        end
        # store last nonzero column (for additional row reduction on B later)
        last_nonzero_col = j
        # ... and set A[i, j] as the new column pivot entry 
        pivot = A[i, j]
        # set that column pivot to 1 (in A)
        A[:, j] = A[:, j]/pivot
        # and do the inverse operation on B
        B[j, :] = B[j, :]*pivot
        # annihilate the nonzero elements to the right of the pivot in A
        # and do the opposite transformations on B
        for col in (j+1):n_cols
            scale = A[i, col]
            A[:, col] -= scale*A[:, j]
            B[j, :] += scale*B[col, :]
        end
        # increase row and column indices and search for the next pivot
        i += 1
        j += 1
    end
    # now perform a row reduction on nonzero rows of B 
    # (Performing inverse operations on columns of A would only be required to keep track of the generators...)
    n_rows, n_cols = size(B)
    i, j = last_nonzero_col+1, 1
    last_nonzero_row = last_nonzero_col
    while i <= n_rows && j <= n_cols
        # check if B[i, j] can be used as a column pivot ...
        if B[i, j] == 0
            # if not, find a row pivot in the current column
            nonzero_row = i+1
            while nonzero_row <= n_rows && B[nonzero_row, j] == 0
                nonzero_row += 1
            end
            # if there are only zeros in the current column downwards
            if nonzero_row == n_rows+1
                # go to a new column
                j += 1
                continue
            end
            # otherwise swap the rows of B
            B[[i nonzero_row], :] = B[[nonzero_row i], :]
            # (The corresponding columns of the column-reduced A are 0, no need to swap those...)
        end
        # store last nonzero row (to return the 'Betti number')
        last_nonzero_row = i
        # Set B[i, j] as the new pivot entry ...
        pivot = B[i, j]
        # ... and set that pivot to 1.
        B[i, :] = B[i, :]/pivot
        # annihilate the nonzero elements below the pivot in B
        # (No need for inverse operations on the columns of the reduced A...)
        for row in (i+1):n_rows
            scale = B[row, j]
            B[row, :] -= scale*B[i, :]
        end
        # increase row and column indices and search for the next pivot
        i += 1
        j += 1
    end
    # return the corresponding Betti number
    return n_rows - last_nonzero_row
end

simultaneous_reduce (generic function with 1 method)

In [13]:
MB_edges=MiniBall_edges(pts)
MB_triangles=MiniBall_triangles(pts)
MB_tetrahedra=MiniBall_tetrahedra(pts)
simplex=cech(0.1, pts, MB_edges, MB_triangles, MB_tetrahedra)
betti_numbers(simplex, 2)

3-element Vector{Any}:
 51
  0
  0

In [14]:
function covers_sphere(simplex)
    β = betti_numbers(simplex, 2)
    return β[1] == 1 && β[2] == 0 && β[3] == 1
end

covers_sphere (generic function with 1 method)

In [ ]:

# =========================================
# Step 1: Candidate radii
# =========================================
function candidate_radii(MB_edges, MB_triangles, MB_tetrahedra)
    Rs = Float64[]
    append!(Rs, values(MB_edges))
    append!(Rs, values(MB_triangles))
    append!(Rs, values(MB_tetrahedra))
    sort!(unique!(Rs))
end

function covers_sphere(simplex)
    β = betti_numbers(simplex, 2)
    return β[1] == 1 && β[2] == 0 && β[3] == 1
end

# =========================================
# Step 3: Binary search over candidate radii
# =========================================
function find_covering_R_binary(pts, MB_edges, MB_triangles, MB_tetrahedra)
    Rs = candidate_radii(MB_edges, MB_triangles, MB_tetrahedra)
    println(length(Rs))
    lo = 1
    hi = length(Rs)
    answer = nothing

    while lo <= hi
        mid = (lo + hi) ÷ 2
        R = Rs[mid]
         println("Trying candidate R = ", R); flush(stdout)  # immediate output

        simplex = cech(R, pts, MB_edges, MB_triangles, MB_tetrahedra)
        println("Constructed Čech complex for R = ", R); flush(stdout)
        if R ≥ 0.6 #to big takes to long
            hi = mid - 1
            continue
        end
        if covers_sphere(simplex)
            answer = R
            hi = mid - 1  # try smaller radius
        else
            lo = mid + 1
        end
    end
    return answer
end

# =========================================
# Step 4: Run it
# =========================================
R_star = find_covering_R_binary(pts, MB_edges, MB_triangles, MB_tetrahedra)
println("Minimal covering radius R = ", R_star)

62547
Trying candidate R = 0.9999999171729765
Constructed Čech complex for R = 0.9999999171729765
Trying candidate R = 0.9929130796287193
Constructed Čech complex for R = 0.9929130796287193
Trying candidate R = 0.9024712175146792
Constructed Čech complex for R = 0.9024712175146792
Trying candidate R = 0.788484591553558
Constructed Čech complex for R = 0.788484591553558
Trying candidate R = 0.6821139890659204
Constructed Čech complex for R = 0.6821139890659204
Trying candidate R = 0.5402347950042576
Constructed Čech complex for R = 0.5402347950042576
Trying candidate R = 0.4795472121306202
Constructed Čech complex for R = 0.4795472121306202
Trying candidate R = 0.33216102052543733
Constructed Čech complex for R = 0.33216102052543733
Trying candidate R = 0.43716964043507184
Constructed Čech complex for R = 0.43716964043507184
Trying candidate R = 0.3615848525028043
Constructed Čech complex for R = 0.3615848525028043
Trying candidate R = 0.3422106943778469
Constructed Čech complex for R =

R = 0.35613484552110825